1. 파일 불러오기
2. 텍스트 파일 전처리
3. 최장 단어 문자열 길이, 최다 빈출 단어 횟수 구하기
4. keyword extraction 모델 적용
5. 텍스트 단어 대체
######  https://lovit.github.io/nlp/2018/04/16/krwordrank/

#1. 파일 구글 드라이브로 가져와서 불러오기
- 두 가지 방법 중 하나만 선택

###### 직접 upload

In [ ]:
#내 드라이브에 있는 파일 google drive에 업로드
from google.colab import files
myfile = files.upload()

KeyboardInterrupt: ignored

In [ ]:
#파일 불러오기
import io
import os
import pandas as pd
import numpy as np

texts = open("정보처리산업기사_2과목요점정리.txt","r", encoding='utf-8')

TypeError: ignored

##### google drive mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
filename = '/content/drive/My Drive/OpenSource/data/정보처리산업기사_1과목요점정리.txt'

In [5]:
with open(filename,"r", encoding='utf-8') as text_file:
    texts = text_file.read()

lines = texts.split("\n") #texts를 개행 문자 기준으로 split하여 list로 저장 

for i in range(len(lines)):
    lines[i] = lines[i].replace("\t", " ") #각 문장에서 \t를 띄어쓰기로 대체

#2. 텍스트 파일 전처리
- 네이버 한글 맞춤법 검사기를 바탕으로 맞춤법, 띄어쓰기 올바르게 교정

###### https://wikidocs.net/92961

In [6]:
!pip install git+https://github.com/ssut/py-hanspell.git
from hanspell import spell_checker

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-jg_okee0
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-jg_okee0
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4854 sha256=d334760fc7eadda884e1ee96e419a01d47bb1b8f985c86be3612b0e27c98c1a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-hsuyza4n/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [7]:
#각 줄 별로 맞춤법, 띄어쓰기 교정
for i in range(len(lines)):
    lines[i] = spell_checker.check(lines[i])
    lines[i]  = lines[i].checked
    print(lines[i])

정보시스템 
▪조직체에 필요한 DATA를 수집, 저장해 두었다가 필요시에 처리해서 의사결정에 유용한 정보를 생성하고 분배하는 수단
▪사용하는 목적에 따라 경영정보 시스템, 군사 정보시스템, 인사 행정 정보 시스템, 의사 결정 지원 시스템 등으로 사용됨
정보와 자료
▪자료(Data) : 현실세계에서 관찰이나 측정을 통해 수집한 단순한 사실이나 결괏값으로, 가공되지 않은 상태
▪정보(Information) : 의사 결정에 도움을 줄 수 있는 유용한 형태로, 자료를 가공(처리) 해서 얻을 수 있는 결과
                   자료 처리 시스템
   자료  →                              →   정보

▪자료처리 시스템 : 정보시스템이 사용할 자료를 처리하는 정보 시스템의 서브시스템으로, 처리 형태에 따라 일괄 처리 시스템, 온라인 실시간 처리 시스템, 분산 처리 시스템으로 분류됨
▪데이터 웨어 하우스(DataWare House) : 조직이나 기업체의 중심이 되는 주요 업무 시스템에서 추출되어 새로이 생성된 데이터베이스로서 의사결정지원 시스템을 지원하는 주체적, 통합적, 시간적 데이터의 집합체


▪통합된 데이터(Integrated data) : 자료의 중복을 배제한 데이터의 모임
▪저장된 데이터(Stored data) : 컴퓨터가 접근할 수 있는 저장 매체에 저장된 자료
▪운영 데이터(Operational data) : 조직의 업무를 수행하는 데 있어서 존재 가치가 확실하고 없어서는 안 될 반드시 필요한 자료
▪공용 데이터 : 여러 응용 시스템들이 공동으로 소유하고 유지하는 자료


▪실시간 접근성 : 수시적이고 비정형적인 질의(조회)에 대하여 실시간 처리(real time processing) 응답이 가능함
▪계속적인 변화 : 새로운 데이터의 삽입(insertion), 삭제(deletion), 갱신(update)으로 항상 최신의 데이터를 유지함
▪동시 공용 : 여러 사용자가 동시에 자기가 원하는 데이터를 이용할 수 있음
▪내용에 

# 3. 최장 단어 문자열 길이, 최다 빈출 단어 횟수 구하기

##### 단어의 최소 출현 빈도수와 단어의 최대 길이를 결정짓기 위함
https://stricky.tistory.com/87 

In [9]:
#문장속 명사만 추출
#konlpy의 tag
!pip install konlpy
from konlpy.tag import Twitter
from collections import Counter

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 11.6MB/s 
     |████████████████████████████████| 460kB 43.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [10]:
nlpy = Twitter()
nouns_word = []

for i in lines:
  nouns_word = nouns_word + nlpy.nouns(i)

print(len(nouns_word))

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


3828


In [11]:
# 명사의 빈도수 계산
from collections import Counter
count = Counter(nouns_word)

tag_count = []
tags = []

#길이가 2부터 49까지인 단어 
for n, c in count.most_common():
  dics = {'tag': n, 'count': c}
  if len(dics['tag']) >= 2 and len(tags) <= 49:
    tag_count.append(dics)
    tags.append(dics['tag'])

In [12]:
for tag in tag_count:
  print(" {:<14}".format(tag['tag']), end='\t')
  print("{}".format(tag['count']))

 데이터           	128
 속성            	75
 데이터베이스        	55
 릴레이           	51
 사용            	50
 테이블           	48
 관계            	45
 구성            	39
 기본            	34
 시스템           	33
 정의            	32
 삭제            	31
 개체            	31
 저장            	30
 노드            	30
 조건            	28
 레코드           	28
 구역            	28
 구조            	26
 처리            	24
 삽입            	24
 사용자           	23
 정보            	22
 자료            	22
 큐플            	22
 성명            	21
 제어            	20
 스키마           	20
 정렬            	20
 이용            	19
 언어            	19
 논리            	18
 모델            	18
 타입            	18
 검색            	18
 대한            	17
 파일            	16
 표현            	16
 오버플로          	16
 기억            	14
 공간            	14
 조작            	14
 물리            	14
 프로그램          	14
 기술            	14
 간의            	14
 접근            	13
 모든            	13
 하나            	13
 개념            	13


In [13]:
import pandas as pd
noun_count_df = pd.DataFrame(tag_count)
noun_count_df

,tag,count
0,데이터,128
1,속성,75
2,데이터베이스,55
3,릴레이,51
4,사용,50
5,테이블,48
6,관계,45
7,구성,39
8,기본,34
9,시스템,33


In [14]:
#명사 중 가장 긴 단어의 문자열 길이 
print("최장 단어 길이 ", max(noun_count_df["tag"].str.len()))
#명사 중 최소 출현 빈도 수
print("최소 출현 빈도 수 ", min(noun_count_df["count"]))

최장 단어 길이  6
최소 출현 빈도 수  13


##### 불필요한 문자 제거 (영어, 숫자 옵션 선택 가능) 

In [15]:
!pip install krwordrank
from krwordrank.hangle import normalize

#normalize는 불필요한 특수 기호를 제거
#english=True, number =True를 입력하면 한글, 영어, 숫자를 제외한 다른 글자들을 제거함.
#default는 english =False, number= False
lines = [normalize(each_line, english=True, number=True) for each_line in lines]

lines

['정보시스템',
 '조직체에 필요한 DATA를 수집 저장해 두었다가 필요시에 처리해서 의사결정에 유용한 정보를 생성하고 분배하는 수단',
 '사용하는 목적에 따라 경영정보 시스템 군사 정보시스템 인사 행정 정보 시스템 의사 결정 지원 시스템 등으로 사용됨',
 '정보와 자료',
 '자료 Data 현실세계에서 관찰이나 측정을 통해 수집한 단순한 사실이나 결괏값으로 가공되지 않은 상태',
 '정보 Information 의사 결정에 도움을 줄 수 있는 유용한 형태로 자료를 가공 처리 해서 얻을 수 있는 결과',
 '자료 처리 시스템',
 '자료 정보',
 '',
 '자료처리 시스템 정보시스템이 사용할 자료를 처리하는 정보 시스템의 서브시스템으로 처리 형태에 따라 일괄 처리 시스템 온라인 실시간 처리 시스템 분산 처리 시스템으로 분류됨',
 '데이터 웨어 하우스 DataWare House 조직이나 기업체의 중심이 되는 주요 업무 시스템에서 추출되어 새로이 생성된 데이터베이스로서 의사결정지원 시스템을 지원하는 주체적 통합적 시간적 데이터의 집합체',
 '',
 '',
 '통합된 데이터 Integrated data 자료의 중복을 배제한 데이터의 모임',
 '저장된 데이터 Stored data 컴퓨터가 접근할 수 있는 저장 매체에 저장된 자료',
 '운영 데이터 Operational data 조직의 업무를 수행하는 데 있어서 존재 가치가 확실하고 없어서는 안 될 반드시 필요한 자료',
 '공용 데이터 여러 응용 시스템들이 공동으로 소유하고 유지하는 자료',
 '',
 '',
 '실시간 접근성 수시적이고 비정형적인 질의 조회 에 대하여 실시간 처리 real time processing 응답이 가능함',
 '계속적인 변화 새로운 데이터의 삽입 insertion 삭제 deletion 갱신 update 으로 항상 최신의 데이터를 유지함',
 '동시 공용 여러 사용자가 동시에 자기가 원하는 데이터를 이용할 수 있음',
 '내용에 의한 참조 데이터베이스에 있는 데이터를 참조할 때 

#Keyword

In [16]:
from krwordrank.word import KRWordRank

wordrank_extractor = KRWordRank(
    min_count = min(noun_count_df["count"]), # 단어의 최소 출현 빈도수
    max_length = max(noun_count_df["tag"].str.len()), # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

#keywords는 filtering이 적용된 L parts
#rank는 substriing graph의 substring에 대한 구마
#graph는 substring graph
keywords, rank, graph = wordrank_extractor.extract(lines, beta, max_iter)

scan vocabs ... 
num vocabs = 307
done


In [17]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:100]:
    print('%8s:\t%.4f' % (word, r))

     데이터:	6.6928
      관계:	2.5557
      있는:	2.2781
      기본:	2.2535
    릴레이션:	2.1677
      속성:	2.1328
      사용:	1.9840
     테이블:	1.8868
      구역:	1.8577
     시스템:	1.8380
      개체:	1.8261
      응용:	1.7330
      정의:	1.5697
      있음:	1.5190
      구조:	1.4185
     큐플을:	1.4184
     논리적:	1.3898
      삽입:	1.3227
      삭제:	1.2365
      언어:	1.2144
     SQL:	1.1844
      저장:	1.1808
      조작:	1.1737
      모든:	1.1610
      같은:	1.1469
      구성:	1.1456
      순차:	1.1371
      노드:	1.1029
      간의:	1.0996
      대한:	1.0809
     레코드:	1.0635
      제어:	1.0548
      개념:	1.0345
      정보:	1.0211
      자료:	1.0193
      처리:	1.0192
      조건:	0.9522
      타입:	0.9493
    오버플로:	0.9434
     물리적:	0.9330
      파일:	0.9114
     스키마:	0.9024
      이상:	0.8689
      기억:	0.8511
      모델:	0.8475
      공간:	0.8065
      이용:	0.7884
      접근:	0.7640
    프로그램:	0.7374
      검색:	0.7246
      관리:	0.7171
      정렬:	0.6507
      하나:	0.6100
      In:	0.5956
      표현:	0.5948
      기술:	0.5804
      연산:	0.5614
      DB:	0.4215
      없음:	0.42

# 텍스트에서 위의 주요 단어 빈칸으로 대체

https://www.delftstack.com/ko/howto/python/how-to-check-whether-a-string-contains-substring-in-python/

In [18]:
keywords_list = keywords.keys()
keywords_list

dict_keys(['데이터', '관계', '있는', '기본', '릴레이션', '속성', '사용', '테이블', '구역', '시스템', '개체', '응용', '정의', '있음', '구조', '큐플을', '논리적', '삽입', '삭제', '언어', 'SQL', '저장', '조작', '모든', '같은', '구성', '순차', '노드', '간의', '대한', '레코드', '제어', '개념', '정보', '자료', '처리', '조건', '타입', '오버플로', '물리적', '파일', '스키마', '이상', '기억', '모델', '공간', '이용', '접근', '프로그램', '검색', '관리', '정렬', '하나', 'In', '표현', '기술', '연산', 'DB', '없음', '제거', 'Le'])

In [19]:
# 입력받은 txt 파일 읽기 모드로 재호출
with open(filename,"r", encoding='utf-8') as text_file:
    new_text = text_file.read()

new_lines = new_text.split("\n") #texts를 개행 문자 기준으로 split하여 list로 저장 

for i in range(len(new_lines)):
    new_lines[i] = new_lines[i].replace("\t", " ") #각 문장에서 \t를 띄어쓰기로 대체

In [29]:
new_text = []
for each_line in new_lines :
  for word in keywords_list: 
    if word in each_line: 
      each_line = each_line.replace(word,"□"*len(word))
  new_text.append(each_line)

In [30]:
new_text

['□□□□□ ',
 '▪조직체에 필요한 DATA를 수집, □□해 두었다가 필요시에 □□해서 의사결정에 유용한 □□를 생성하고 분배하는 수단',
 '▪□□하는 목적에 따라 경영□□ □□□, 군사 □□□□□, 인사 행정 □□ □□□, 의사 결정 지원 □□□ 등으로 □□됨',
 '□□와 □□',
 '▪□□(Data) : 현실세계에서 관찰이나 측정을 통해 수집한 단순한 사실이나 결과값으로, 가공되지 않은 상태',
 '▪□□(□□formation) : 의사 결정에 도움을 줄 수 □□ 유용한 형태로, □□를 가공(□□)해서 얻을 수 □□ 결과',
 '                   □□ □□ □□□',
 '   □□  →                              →   □□',
 '',
 '▪□□□□ □□□ : □□□□□이 □□할 □□를 □□하는 □□ □□□의 서브□□□으로, □□형태에 따라 일괄 □□ □□□, 온라인 실시간 □□ □□□, 분산 □□ □□□으로 분류됨',
 '▪□□□ 웨어 하우스(DataWare House) : 조직이나 기업체의 중심이 되는 주요 업무 □□□에서 추출되어 새로이 생성된 □□□베이스로서 의사결정지원 □□□을 지원하는 주체적, 통합적, 시간적 □□□의 집합체',
 '',
 '',
 '▪통합된 □□□(□□tegrated data) : □□의 중복을 배제한 □□□의 모임',
 '▪□□된 □□□(Stored data) : 컴퓨터가 □□할 수 □□ □□ 매체에 □□된 □□',
 '▪운영 □□□(Operational data) : 조직의 업무를 수행하는데 있어서 존재 가치가 확실하고 없어서는 안 될 반드시 필요한 □□',
 '▪공용 □□□ : 여러 □□ □□□들이 공동으로 소유하고 유지하는 □□',
 '',
 '',
 '▪실시간 □□성 : 수시적이고 비정형적인 질의(조회)에 대하여 실시간 □□(real time processing) 응답이 가능함',
 '▪계속적인 변화 : 새로운 □□□의 □□(insertion), □□(deletion)